## airtable_tools_v2

In [22]:
import os, logging
import pandas as pd
from pyairtable import Table
from dotenv import load_dotenv
from datetime import datetime
import pytz

In [25]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    datefmt='%m-%d %H:%M:%S',level='INFO')
logger=logging.getLogger(__name__)
logger.setLevel('INFO')

In [26]:
load_dotenv()
api_key=os.getenv('AIRTABLE_API_KEY')
base_id='appDTdyxcShkSR3oF'

In [28]:
action_table=Table(api_key, base_id, 'Log')

In [29]:
action_table.all()

[{'id': 'recMHleWhG91jn7sc',
  'createdTime': '2022-05-08T19:00:41.000Z',
  'fields': {'Date': '2022-05-07',
   'Habit': ['recFqsNfUNIe88oRK'],
   'Score': 5,
   'Goal 2': ['recenm1UYUwAIcoya']}},
 {'id': 'recWwwlzwY1XqGQdE',
  'createdTime': '2022-05-08T21:56:53.000Z',
  'fields': {'Date': '2022-05-08',
   'Habit': ['recFqsNfUNIe88oRK'],
   'Score': 5,
   'Goal 2': ['recenm1UYUwAIcoya']}},
 {'id': 'recmIsNssDDyEl9io',
  'createdTime': '2022-05-08T21:56:49.000Z',
  'fields': {'Date': '2022-05-08',
   'Habit': ['rec24NGEkbvktkD9B'],
   'Score': 4,
   'Goal 2': ['recenm1UYUwAIcoya']}},
 {'id': 'recqlcZ0qzaBZ7EHW',
  'createdTime': '2022-05-08T17:11:28.000Z',
  'fields': {'Date': '2022-05-07',
   'Habit': ['rec24NGEkbvktkD9B'],
   'Score': 3,
   'Goal 2': ['recenm1UYUwAIcoya']}}]

In [ ]:
habit_table=Table(api_key, base_id, 'Habits')

In [37]:
for o in action_table.all():
    if len(o['fields']['Habit'])>0:
        habit=habit_table.get(o['fields']['Habit'][0])
        if 'Log' in habit['fields']['Habit']:
            
            

In [43]:
res=action_table.all()
logger.info('Found %i records', len(res))
res=[{**r['fields'], **r} for r in res]

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)

[05-08 20:59:44] p1144 {<ipython-input-43-f504cc177592>:2} INFO - Found 4 records
[05-08 20:59:44] p1144 {<ipython-input-43-f504cc177592>:5} INFO - Converting to dataframe and resampling...


In [44]:
df

,Date,Habit,Score,Goal 2,id,createdTime,fields
0,2022-05-07,[recFqsNfUNIe88oRK],5,[recenm1UYUwAIcoya],recMHleWhG91jn7sc,2022-05-08T19:00:41.000Z,"{'Date': '2022-05-07', 'Habit': ['recFqsNfUNIe..."
1,2022-05-08,[recFqsNfUNIe88oRK],5,[recenm1UYUwAIcoya],recWwwlzwY1XqGQdE,2022-05-08T21:56:53.000Z,"{'Date': '2022-05-08', 'Habit': ['recFqsNfUNIe..."
2,2022-05-08,[rec24NGEkbvktkD9B],4,[recenm1UYUwAIcoya],recmIsNssDDyEl9io,2022-05-08T21:56:49.000Z,"{'Date': '2022-05-08', 'Habit': ['rec24NGEkbvk..."
3,2022-05-07,[rec24NGEkbvktkD9B],3,[recenm1UYUwAIcoya],recqlcZ0qzaBZ7EHW,2022-05-08T17:11:28.000Z,"{'Date': '2022-05-07', 'Habit': ['rec24NGEkbvk..."


In [ ]:
df.index=pd.DatetimeIndex(df['Date'])
df=df.sort_index()
dfr=df.resample('D').sum() #Resample

In [31]:
goal_table=Table(api_key, base_id, 'Goals')


In [35]:
goal_table.get('recenm1UYUwAIcoya')['fields']['Goal']

'Personal Development'